In [0]:
from pyspark.sql.functions import *


list = [
("T1","B","Pune","Mumbai"),
("T2","B","Mumbai","Pune"),
("T3","M","Pune","Delhi"),
("T4","D","Mumbai","Mumbai"),
("T5","E","Mumbai","Pune"),
("T6","E","Pune","Delhi")
]

df = spark.createDataFrame(list,["time","name","source","destination"])
df.show()
df.createOrReplaceTempView("data")


In [0]:
%sql

select *,row_number() over( partition by name order by time) as rownum,
case when rownum=1 then source else destination end as source1,
case when rownum=1 then destination else source end as destination1
from data c 

In [0]:
%sql
with base as (
select *,row_number() over( partition by name order by time) as rownum,
case when rownum=1 then source else destination end as source1,
case when rownum=1 then destination else source end as destination1
from data c 
), final as (
select distinct name,source1,destination1,case when source1=destination1 then 0 else 1 end as flag
from base b1
)
select name , sum(flag)
from final 
group by name
